In [1]:
import pandas as pd
import numpy as np

In [2]:
class LibffmConverter:
    """Converts an input dataframe to another dataframe in libffm format. A text file of the converted
    Dataframe is optionally generated.
    .. note::
        The input dataframe is expected to represent the feature data in the following schema:
        .. code-block:: python
            |field-1|field-2|...|field-n|rating|
            |feature-1-1|feature-2-1|...|feature-n-1|1|
            |feature-1-2|feature-2-2|...|feature-n-2|0|
            ...
            |feature-1-i|feature-2-j|...|feature-n-k|0|
        Where
        1. each `field-*` is the column name of the dataframe (column of label/rating is excluded), and
        2. `feature-*-*` can be either a string or a numerical value, representing the categorical variable or
        actual numerical variable of the feature value in the field, respectively.
        3. If there are ordinal variables represented in int types, users should make sure these columns
        are properly converted to string type.
        The above data will be converted to the libffm format by following the convention as explained in
        `this paper <https://www.csie.ntu.edu.tw/~r01922136/slides/ffm.pdf>`_.
        i.e. `<field_index>:<field_feature_index>:1` or `<field_index>:<field_index>:<field_feature_value>`, depending on
        the data type of the features in the original dataframe.
    Args:
        filepath (str): path to save the converted data.
    Attributes:
        field_count (int): count of field in the libffm format data
        feature_count (int): count of feature in the libffm format data
        filepath (str or None): file path where the output is stored - it can be None or a string
    Examples:
        >>> import pandas as pd
        >>> df_feature = pd.DataFrame({
                'rating': [1, 0, 0, 1, 1],
                'field1': ['xxx1', 'xxx2', 'xxx4', 'xxx4', 'xxx4'],
                'field2': [3, 4, 5, 6, 7],
                'field3': [1.0, 2.0, 3.0, 4.0, 5.0],
                'field4': ['1', '2', '3', '4', '5']
            })
        >>> converter = LibffmConveter().fit(df_feature, col_rating='rating')
        >>> df_out = converter.transform(df_feature)
        >>> df_out
            rating field1 field2   field3 field4
        0       1  1:1:1  2:4:3  3:5:1.0  4:4:1
        1       0  1:2:1  2:4:4  3:5:2.0  4:5:1
        2       0  1:3:1  2:4:5  3:5:3.0  4:6:1
        3       1  1:3:1  2:4:6  3:5:4.0  4:7:1
        4       1  1:3:1  2:4:7  3:5:5.0  4:8:1
    """

    def __init__(self, filepath=None):
        self.filepath = filepath
        self.col_rating = None
        self.field_names = None
        self.field_count = None
        self.feature_count = None

    def fit(self, df, col_rating=3):
        """Fit the dataframe for libffm format.
        This method does nothing but check the validity of the input columns
        Args:
            df (pd.DataFrame): input Pandas dataframe.
            col_rating (str): rating of the data.
        Return:
            obj: the instance of the converter
        """

        # Check column types.
        types = df.dtypes
        if not all(
            [
                x == object or np.issubdtype(x, np.integer) or x == np.float
                for x in types
            ]
        ):
            raise TypeError("Input columns should be only object and/or numeric types.")

        if col_rating not in df.columns:
            raise TypeError(
                "Column of {} is not in input dataframe columns".format(col_rating)
            )

        self.col_rating = col_rating
        self.field_names = list(df.drop(col_rating, axis=1).columns)

        return self

    def transform(self, df):
        """Tranform an input dataset with the same schema (column names and dtypes) to libffm format 
        by using the fitted converter.
        Args: 
            df (pd.DataFrame): input Pandas dataframe.
        Return:
            pd.DataFrame: output libffm format dataframe.
        """
        if self.col_rating not in df.columns:
            raise ValueError(
                "Input dataset does not contain the label column {} in the fitting dataset".format(
                    self.col_rating
                )
            )

        if not all([x in df.columns for x in self.field_names]):
            raise ValueError(
                "Not all columns in the input dataset appear in the fitting dataset"
            )

        # Encode field-feature.
        idx = 1
        self.field_feature_dict = {}
        for field in self.field_names:
            for feature in df[field].values:
                # Check whether (field, feature) tuple exists in the dict or not.
                # If not, put them into the key-values of the dict and count the index.
                if (field, feature) not in self.field_feature_dict:
                    self.field_feature_dict[(field, feature)] = idx
                    if df[field].dtype == object:
                        idx += 1
            if df[field].dtype != object:
                idx += 1

        self.field_count = len(self.field_names)
        self.feature_count = idx - 1

        def _convert(field, feature, field_index, field_feature_index_dict):
            field_feature_index = field_feature_index_dict[(field, feature)]
            if isinstance(feature, str):
                feature = 1
            return "{}:{}:{}".format(field_index, field_feature_index, feature)

        for col_index, col in enumerate(self.field_names):
            df[col] = df[col].apply(
                lambda x: _convert(col, x, col_index + 1, self.field_feature_dict)
            )

        # Move rating column to the first.
        column_names = self.field_names[:]
        column_names.insert(0, self.col_rating)
        df = df[column_names]

        if self.filepath is not None:
            np.savetxt(self.filepath, df.values, delimiter=" ", fmt="%s")

        return df

    def fit_transform(self, df, col_rating=4):
        """Do fit and transform in a row
        Args:
            df (pd.DataFrame): input Pandas dataframe.
            col_rating (str): rating of the data.
        Return:
            pd.DataFrame: output libffm format dataframe.
        """
        return self.fit(df, col_rating=col_rating).transform(df)

    def get_params(self):
        """Get parameters (attributes) of the libffm converter
        Return:
            dict: parameters field count, feature count, and file path.
        """
        return {
            "field count": self.field_count,
            "feature count": self.feature_count,
            "file path": self.filepath,
        }


    def negative_feedback_sampler(
        df,
        col_user=0,
        col_item=1,
        col_label=1,
        ratio_neg_per_user=1,
        seed=42,
    ):
        """Utility function to sample negative feedback from user-item interaction dataset.
        This negative sampling function will take the user-item interaction data to create 
        binarized feedback, i.e., 1 and 0 indicate positive and negative feedback, 
        respectively. 
        Negative sampling is used in the literature frequently to generate negative samples 
        from a user-item interaction data.
        See for example the `neural collaborative filtering paper <https://www.comp.nus.edu.sg/~xiangnan/papers/ncf.pdf>`_.

        Args:
            df (pandas.DataFrame): input data that contains user-item tuples.
            col_user (str): user id column name.
            col_item (str): item id column name.
            col_label (str): label column name. It is used for the generated columns where labels
            of positive and negative feedback, i.e., 1 and 0, respectively, in the output dataframe.
            ratio_neg_per_user (int): ratio of negative feedback w.r.t to the number of positive feedback for each user. 
            If the samples exceed the number of total possible negative feedback samples, it will be reduced to the number
            of all the possible samples.
            seed (int): seed for the random state of the sampling function.
        Returns:
            pandas.DataFrame: data with negative feedback 
        Examples:
            >>> import pandas as pd
            >>> df = pd.DataFrame({
                'userID': [1, 2, 3],
                'itemID': [1, 2, 3],
                'rating': [5, 5, 5]
            })
            >>> df_neg_sampled = negative_feedback_sampler(
                df, col_user='userID', col_item='itemID', ratio_neg_per_user=1
            )
            >>> df_neg_sampled
            userID  itemID  feedback
            1   1   1
            1   2   0
            2   2   1
            2   1   0
            3   3   1
            3   1   0
        """
        # Get all of the users and items.
        users = df[col_user].unique()
        items = df[col_item].unique()

        # Create a dataframe for all user-item pairs
        df_neg = user_item_pairs(
            pd.DataFrame(users, columns=[col_user]),
            pd.DataFrame(items, columns=[col_item]),
            user_item_filter_df=df,
        )
        df_neg[col_label] = 0

        df_pos = df.copy()
        df_pos[col_label] = 1

        df_all = pd.concat([df_pos, df_neg], ignore_index=True, sort=True)
        df_all = df_all[[col_user, col_item, col_label]]

        # Sample negative feedback from the combined dataframe.
        df_sample = (
            df_all.groupby(col_user)
            .apply(
                lambda x: pd.concat(
                    [
                        x[x[col_label] == 1],
                        x[x[col_label] == 0].sample(
                            min(
                                max(
                                    round(len(x[x[col_label] == 1]) * ratio_neg_per_user), 1
                                ),
                                len(x[x[col_label] == 0]),
                            ),
                            random_state=seed,
                            replace=False,
                        )
                        if len(x[x[col_label] == 0] > 0)
                        else pd.DataFrame({}, columns=[col_user, col_item, col_label]),
                    ],
                    ignore_index=True,
                    sort=True,
                )
            )
            .reset_index(drop=True)
            .sort_values(col_user)
        )

        return df_sample


    def has_columns(df, columns):
        """Check if DataFrame has necessary columns
        Args:
            df (pd.DataFrame): DataFrame
            columns (list(str): columns to check for
        Returns:
            bool: True if DataFrame has specified columns
        """

        result = True
        for column in columns:
            if column not in df.columns:
                logger.error("Missing column: {} in DataFrame".format(column))
                result = False

        return result


    def has_same_base_dtype(df_1, df_2, columns=None):
        """Check if specified columns have the same base dtypes across both DataFrames
        Args:
            df_1 (pd.DataFrame): first DataFrame
            df_2 (pd.DataFrame): second DataFrame
            columns (list(str)): columns to check, None checks all columns
        Returns:
            bool: True if DataFrames columns have the same base dtypes
        """

        if columns is None:
            if any(set(df_1.columns).symmetric_difference(set(df_2.columns))):
                logger.error(
                    "Cannot test all columns because they are not all shared across DataFrames"
                )
                return False
            columns = df_1.columns

        if not (
            has_columns(df=df_1, columns=columns) and has_columns(df=df_2, columns=columns)
        ):
            return False

        result = True
        for column in columns:
            if df_1[column].dtype.type.__base__ != df_2[column].dtype.type.__base__:
                logger.error("Columns {} do not have the same base datatype".format(column))
                result = False

        return result

In [3]:
all_features = ["engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "enaging_user_follower_count", "enaging_user_following_count"]
train = pd.read_csv("/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/like/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [10, 11, 15, 16], sep="\x01")

test = pd.read_csv("/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/like/competition_test.tsv", encoding="utf-8",
                    names = all_features, usecols= [10, 11, 15, 16], sep="\x01")

train_test_combined = pd.concat([train,test],keys=[0,1])

from sklearn import preprocessing

x = train_test_combined.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.index = train_test_combined.index
train_test_combined_scaled = pd.concat([train_test_combined, df], axis = 1)
train_scaled, test_scaled = train_test_combined_scaled.xs(0),train_test_combined_scaled.xs(1)
train_scaled_only = train_scaled[[0, 1, 2, 3]]
test_scaled_only = test_scaled[[0, 1, 2, 3]]
train_scaled_only.columns = ['a', 'b', 'c', 'd']
test_scaled_only.columns = ['a', 'b', 'c', 'd']

In [4]:
all_features = ["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_is_verified",\
                "engagee_follows_engager", "like_timestamp"]

train = pd.read_csv("/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/like/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 9, 12, 14, 17, 19, 23], sep="\x01")
train['like_bool'] = train.like_timestamp.fillna(0)
train.loc[train.like_bool != 0.0, 'like_bool'] = 1.0
train = train[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_is_verified",
               "engagee_follows_engager", "like_bool"]]

In [5]:
all_features = ["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_is_verified",\
                "engagee_follows_engager", "like_timestamp"]

test = pd.read_csv("/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/like/competition_test.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 9, 12, 14, 17, 19, 23], sep="\x01")
test['like_bool'] = test.like_timestamp.fillna(0)
test.loc[test.like_bool != 0.0, 'like_bool'] = 1.0
test = test[["tweet_type", "Language", "enaged_with_user_id",
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_is_verified", 
             "engagee_follows_engager", "like_bool"]]

In [6]:
train_df = pd.concat([train_scaled_only, train], axis = 1)
test_df = pd.concat([test_scaled_only, test], axis = 1)

In [7]:
train_df.head()

,a,b,c,d,tweet_type,Language,enaged_with_user_id,engaged_with_user_is_verified,engaging_user_id,enaging_user_is_verified,engagee_follows_engager,like_bool
0,0.000031,0.000133,2.522795e-04,0.000317,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,False,0000006C3074607050F1339DDCB890BB,False,False,0.0
1,0.000352,0.007416,1.563640e-07,0.000040,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,False,000005C520010F8917EEAB6F5B6EC1C4,False,True,0.0
2,0.001255,0.000727,1.839576e-08,0.000015,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,False,000006829BEADA9EEA695CF0C334B426,False,False,1.0
3,0.000027,0.000022,2.759365e-08,0.000015,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,False,000006829BEADA9EEA695CF0C334B426,False,False,0.0
4,0.001251,0.000727,2.759365e-08,0.000015,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,False,000006829BEADA9EEA695CF0C334B426,False,False,0.0


In [8]:
test_df.head()

,a,b,c,d,tweet_type,Language,enaged_with_user_id,engaged_with_user_is_verified,engaging_user_id,enaging_user_is_verified,engagee_follows_engager,like_bool
0,0.000001,0.000028,1.020965e-06,0.000356,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,6720CC7830F94CB7465CA283300DB010,False,00000776B07587ECA9717BFC301F2D6E,False,True,0.0
1,0.000002,0.000059,1.020965e-06,0.000356,Retweet,22C448FF81263D4BAF2A176145EE9EAD,7DDC67265CFB6E0B4820E0BD0E33A8D3,False,00000776B07587ECA9717BFC301F2D6E,False,True,0.0
2,0.000038,0.000148,1.057756e-05,0.000025,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,5456A10C7E4F7A415948EA88BE6845D6,False,00000B85AAF7DE172876FD96718C4469,False,True,0.0
3,0.000002,0.000042,7.183546e-06,0.000234,Retweet,125C57F4FA6D4E110983FB11B52EFD4E,9D421C234C7B59A0EDC8D85C847D4569,False,00000E0C9B364891CDE89ECFC54771DE,False,True,0.0
4,0.000009,0.000472,1.379682e-07,0.000065,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,F63ECD1C7827E767E7C44E9A717056AC,False,0000109A57AFA64758EE4AAE2A01BFC7,False,False,0.0


In [9]:
data = pd.concat([train_df,test_df], keys=[0,1])

In [10]:
data.head()

a         b             c         d tweet_type  \
0 0  0.000031  0.000133  2.522795e-04  0.000317   TopLevel   
  1  0.000352  0.007416  1.563640e-07  0.000040   TopLevel   
  2  0.001255  0.000727  1.839576e-08  0.000015   TopLevel   
  3  0.000027  0.000022  2.759365e-08  0.000015   TopLevel   
  4  0.001251  0.000727  2.759365e-08  0.000015   TopLevel   

                             Language               enaged_with_user_id  \
0 0  22C448FF81263D4BAF2A176145EE9EAD  D7EFC6D19CFB09B9A1CE4C4B0E0091F6   
  1  B9175601E87101A984A50F8A62A1C374  1F30A85E612A24B426013978ADB7AE46   
  2  22C448FF81263D4BAF2A176145EE9EAD  9E1FD934C376903895DEEBCC0F6AB920   
  3  22C448FF81263D4BAF2A176145EE9EAD  BEB3C337B266E2C4769065AD64724329   
  4  22C448FF81263D4BAF2A176145EE9EAD  9E1FD934C376903895DEEBCC0F6AB920   

     engaged_with_user_is_verified                  engaging_user_id  \
0 0                          False  0000006C3074607050F1339DDCB890BB   
  1                          False  000005C520010F8917EEAB6F5B6EC1C4   
  2                          False  000006829BEADA9EEA695CF0C334B426   
  3                          False  000006829BEADA9EEA695CF0C334B426   
  4                          False  000006829BEADA9EEA695CF0C334B426   

     enaging_user_is_verified  engagee_follows_engager  like_bool  
0 0                     False                    False        0.0  
  1                     False                     True        0.0  
  2                     False                    False        1.0  
  3                     False                    False        0.0  
  4                     False                    False        0.0

In [11]:
data.dtypes

a                                float64
b                                float64
c                                float64
d                                float64
tweet_type                        object
Language                          object
enaged_with_user_id               object
engaged_with_user_is_verified       bool
engaging_user_id                  object
enaging_user_is_verified            bool
engagee_follows_engager             bool
like_bool                        float64
dtype: object

In [12]:
data['engaged_with_user_is_verified'] = data['engaged_with_user_is_verified'].astype(str)
data['enaging_user_is_verified'] = data['enaging_user_is_verified'].astype(str)
data['engagee_follows_engager'] = data['engagee_follows_engager'].astype(str)

In [13]:
converter = LibffmConverter().fit(data, col_rating='like_bool')

In [14]:
del[[train_df, test_df, train, test, train_scaled, test_scaled, train_scaled_only, test_scaled_only]]
import gc
gc.collect()
train_df = pd.DataFrame()
test_df = pd.DataFrame()
train = pd.DataFrame()
test = pd.DataFrame()
train_scaled = pd.DataFrame()
test_scaled = pd.DataFrame()
train_scaled_only = pd.DataFrame()
test_scaled_only = pd.DataFrame()

In [15]:
df_out = converter.transform(data)

In [16]:
train_ffm, test_ffm = df_out.xs(0),df_out.xs(1)

In [17]:
train_ffm

,like_bool,a,b,c,d,tweet_type,Language,enaged_with_user_id,engaged_with_user_is_verified,engaging_user_id,enaging_user_is_verified,engagee_follows_engager
0,0.0,1:1:3.112473946641131e-05,2:2:0.00013267263858444142,3:3:0.00025227951576976,4:4:0.00031726375634990965,5:5:1,6:8:1,7:74:1,8:15702463:1,9:15702465:1,10:42963993:1,11:42963995:1
1,0.0,1:1:0.0003521893514423134,2:2:0.007415725890233405,3:3:1.5636399912811435e-07,4:4:4.0253832191307404e-05,5:5:1,6:9:1,7:75:1,8:15702463:1,9:15702466:1,10:42963993:1,11:42963996:1
2,1.0,1:1:0.0012545943404087437,2:2:0.0007267762167879909,3:3:1.839576460330757e-08,4:4:1.483035922837641e-05,5:5:1,6:8:1,7:76:1,8:15702463:1,9:15702467:1,10:42963993:1,11:42963995:1
3,0.0,1:1:2.693759448131104e-05,2:2:2.2262019016711357e-05,3:3:2.7593646904961354e-08,4:4:1.5360014915104137e-05,5:5:1,6:8:1,7:77:1,8:15702463:1,9:15702467:1,10:42963993:1,11:42963995:1
4,0.0,1:1:0.0012514429162761946,2:2:0.000727450823424861,3:3:2.7593646904961354e-08,4:4:1.5360014915104137e-05,5:5:1,6:8:1,7:76:1,8:15702463:1,9:15702467:1,10:42963993:1,11:42963995:1
...,...,...,...,...,...,...,...,...,...,...,...,...
148075233,0.0,1:1:0.0002017708153179502,2:2:0.005124087144785875,3:3:2.6591077734081094e-05,4:4:0.0022393842434848383,5:6:1,6:11:1,7:1906238:1,8:15702463:1,9:27833876:1,10:42963993:1,11:42963996:1
148075234,0.0,1:1:1.073785806961232e-05,2:2:0.00029592744470699137,3:3:2.6618671380986055e-05,4:4:0.0022388545877981103,5:6:1,6:11:1,7:4636551:1,8:15702463:1,9:27833876:1,10:42963993:1,11:42963995:1
148075235,1.0,1:1:0.008706499646735092,2:2:9.939204449885272e-05,3:3:2.6618671380986055e-05,4:4:0.0022388545877981103,5:5:1,6:11:1,7:995:1,8:15702464:1,9:27833876:1,10:42963993:1,11:42963995:1
148075236,1.0,1:1:4.439435962003775e-05,2:2:0.001118497803930528,3:3:7.0455778430668e-06,4:4:0.00015995601739177414,5:5:1,6:31:1,7:2023737:1,8:15702463:1,9:38897245:1,10:42963993:1,11:42963996:1


In [18]:
test_ffm

,like_bool,a,b,c,d,tweet_type,Language,enaged_with_user_id,engaged_with_user_is_verified,engaging_user_id,enaging_user_is_verified,engagee_follows_engager
0,0.0,1:1:1.0445731675303e-06,2:2:2.8108609869585045e-05,3:3:1.02096493548357e-06,4:4:0.0003559286214810339,5:5:1,6:8:1,7:9285322:1,8:15702463:1,9:15702468:1,10:42963993:1,11:42963996:1
1,0.0,1:1:1.6642352160652236e-06,2:2:5.936538404456362e-05,3:3:1.02096493548357e-06,4:4:0.0003559286214810339,5:6:1,6:8:1,7:3367747:1,8:15702463:1,9:15702468:1,10:42963993:1,11:42963996:1
2,0.0,1:1:3.816232987477223e-05,2:2:0.00014841346011140903,3:3:1.0577564646901852e-05,4:4:2.489381727620326e-05,5:6:1,6:11:1,7:598658:1,8:15702463:1,9:15702472:1,10:42963993:1,11:42963996:1
3,0.0,1:1:2.3989773593280617e-06,2:2:4.160074260698587e-05,3:3:7.183546077591606e-06,4:4:0.00023357815784692845,5:6:1,6:13:1,7:1678849:1,8:15702463:1,9:15702475:1,10:42963993:1,11:42963996:1
4,0.0,1:1:9.020508963672675e-06,2:2:0.00047155003917215873,3:3:1.3796823452480677e-07,4:4:6.461799378078293e-05,5:6:1,6:11:1,7:5602018:1,8:15702463:1,9:41198553:1,10:42963993:1,11:42963995:1
...,...,...,...,...,...,...,...,...,...,...,...,...
12434833,0.0,1:1:4.770512542221006e-05,2:2:0.00018731577617091475,3:3:5.15081408892612e-07,4:4:0.0001408884126695759,5:6:1,6:10:1,7:1542193:1,8:15702463:1,9:42891073:1,10:42963993:1,11:42963995:1
12434834,0.0,1:1:0.00015709318161858223,2:2:0.0003298826454294501,3:3:1.6896509788138002e-05,4:4:6.77959279011493e-05,5:6:1,6:11:1,7:95876:1,8:15702463:1,9:27833865:1,10:42963993:1,11:42963995:1
12434835,0.0,1:1:3.881740118322343e-05,2:2:1.798951031653443e-05,3:3:7.358305841323028e-08,4:4:2.966071845675282e-05,5:6:1,6:13:1,7:125550:1,8:15702463:1,9:42784662:1,10:42963993:1,11:42963995:1
12434836,0.0,1:1:3.7489553936362884e-05,2:2:0.0009262349124225664,3:3:6.594881610285764e-06,4:4:0.00024523058295493853,5:5:1,6:8:1,7:1146104:1,8:15702463:1,9:27833875:1,10:42963993:1,11:42963996:1


In [19]:
train_ffm.to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/like/train.ffm',\
                 index = False, header = False, sep = ' ')

In [20]:
test_ffm.to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/like/test.ffm',\
                 index = False, header = False, sep = ' ')